# Maven

In [22]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime

def fetch_maven_version_info(group_id: str, artifact_id: str) -> tuple[str, datetime]:
    """
    Fetch the latest version and release date from Maven Central for the given artifact.
    """
    # Convert group_id dots to slashes for URL
    group_path = group_id.replace('.', '/')
    
    url = f"https://repo1.maven.org/maven2/{group_path}/{artifact_id}/maven-metadata.xml"
    response = requests.get(url)
    response.raise_for_status()
    
    # Parse XML
    root = ET.fromstring(response.content)
    
    # Get latest version
    versioning = root.find('versioning')
    if versioning is None:
        raise ValueError(f"No versioning info found for {group_id}:{artifact_id}")
        
    latest = versioning.find('latest')
    if latest is None:
        raise ValueError(f"No latest version found for {group_id}:{artifact_id}")
    
    latest_version = latest.text

    if latest_version is None:
        raise ValueError(f"No latest version found for {group_id}:{artifact_id}")
    
    # Get release date from lastUpdated
    last_updated = versioning.find('lastUpdated')
    if last_updated is None:
        raise ValueError(f"No lastUpdated found for {group_id}:{artifact_id}")
    
    if last_updated.text is None:
        raise ValueError(f"lastUpdated is empty for {group_id}:{artifact_id}")
        
    # Convert Maven timestamp (yyyyMMddHHmmss) to datetime
    timestamp = last_updated.text
    release_dt = datetime.strptime(timestamp, '%Y%m%d%H%M%S')
    
    return latest_version, release_dt

# Example usage
group_id = "org.spkringframework.boot"
artifact_id = "spring-boot-starter-parent"

version, date = fetch_maven_version_info(group_id, artifact_id)
print(f"Latest version: {version}")
print(f"Release date: {date}")

HTTPError: 404 Client Error: Not Found for url: https://repo1.maven.org/maven2/org/spkringframework/boot/spring-boot-starter-parent/maven-metadata.xml

In [ ]:
from urllib.parse import urlencode
import requests
from datetime import datetime, UTC

def fetch_maven_release_date(group_id: str, artifact_id: str, version: str) -> datetime:
    """
    Returns the UTC datetime at which a specific Maven artifact version
    was published to Maven Central, using the search.maven.org API.
    """
    # Build the Solr-style query:
    #   q = g:"groupId" AND a:"artifactId" AND v:"version"
    #   core=gav (search only group/artifact/version data)
    #   rows=1 (just need the first doc)
    #   wt=json (we want JSON response)
    query = f'g:"{group_id}" AND a:"{artifact_id}" AND v:"{version}"'
    url = "https://search.maven.org/solrsearch/select"
    params = {
        "q": query,
        "core": "gav",
        "rows": 1,
        "wt": "json",
    }
    resp = requests.get(url, params=params)

    resp.raise_for_status()
    data = resp.json()

    docs = data.get("response", {}).get("docs", [])
    if not docs: # here raise LibraryVersionNotFoundError
        raise ValueError(
            f"Could not find artifact {group_id}:{artifact_id}:{version} on Maven Central"
        )
    
    doc = docs[0]
    assert doc["g"] == group_id, f"Group mismatch: {doc['g']} != {group_id}"
    assert doc["a"] == artifact_id, f"Artifact mismatch: {doc['a']} != {artifact_id}"
    assert doc["v"] == version, f"Version mismatch: {doc['v']} != {version}"
    
    # 'timestamp' is epoch milliseconds
    ts_millis = docs[0]["timestamp"]
    return datetime.fromtimestamp(ts_millis / 1000.0, UTC)


version = "3.4.3"
group_id = "org.springframework.boot"
artifact_id = "spring-boot-starter-parent"

dt = fetch_maven_release_date(
    group_id=group_id,
    artifact_id=artifact_id,
    version=version,
)
print(f"{group_id}:{artifact_id}:{version} published at: {dt.isoformat()}")


ValueError: Could not find artifact org.springframework.boot:spring-boot-starter-parent:3.4.3 on Maven Central

# C# / Nuget

In [ ]:
https://api.nuget.org/v3/catalog0/index.json

# Ruby

In [ ]:
https://rubygems.org/api/v1/gems/<gem_name>.json

# Go

In [ ]:
pkg.go.dev:

# PHP

In [ ]:
https://packagist.org/packages/<vendor>/<package_name>.json


# Rust

In [ ]:
https://crates.io/api/v1/crates/{crate_name}